In [1]:
import numpy as np
import scipy.sparse as sp
import pickle
import os
import Utils.TimeLogger as logger
from Utils.TimeLogger import log
import Utils.NNLayers as NNs
from Utils.NNLayers import FC, Regularize, Activate, Dropout, Bias, getParam, defineParam
import tensorflow as tf
from tensorflow.core.protobuf import config_pb2
from Params import args
from DataHandler import LoadData, negSamp, transpose, transToLsts

/Users/jingyunliu/opt/anaconda3/envs/tfv1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jingyunliu/opt/anaconda3/envs/tfv1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jingyunliu/opt/anaconda3/envs/tfv1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jingyunliu/opt/ana

In [5]:
def create_norm_adj_mat(self, R):
    '''
    Create normalized adjacency matrix.
    :param R:
        user-item adjacency matrix (scipy matrix)
    :return:
        scipy.sparse.csr_matrix: Normalized adjacency matrix.
    '''
    adj_mat = sp.dok_matrix(
        (args.user + args.item, args.user + args.item), dtype=np.float32
    )
    adj_mat = adj_mat.tolil()
    R = R.tolil()  # R is a user-item adjacency matrix

    adj_mat[:args.user, args.user:] = R
    adj_mat[args.user:, : args.user] = R.T
    adj_mat = adj_mat.todok()
    print("Already create adjacency matrix.")

    rowsum = np.array(adj_mat.sum(1))
    d_inv = np.power(rowsum + 1e-9, -0.5).flatten()
    d_inv[np.isinf(d_inv)] = 0.0
    d_mat_inv = sp.diags(d_inv)
    norm_adj_mat = d_mat_inv.dot(adj_mat)
    norm_adj_mat = norm_adj_mat.dot(d_mat_inv)
    print("Already normalize adjacency matrix.")
    return norm_adj_mat.tocsr()


def _convert_sp_mat_to_sp_tensor(self, X):
    """
    Convert a scipy sparse matrix to tf.SparseTensor.
    :return: tf.SparseTensor: SparseTensor after conversion.
    """
    coo = X.tocoo().astype(np.float32)
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [ ]:
def _create_lightGCN_embed(self, ego_embedding, behavior_ind, n_layers=2):
    #TODO: remember to set n_layers as a self.property!!!
    all_embeddings = [ego_embedding]
    A_hat = (self._convert_sp_mat_to_sp_tensor(self.create_norm_adj_mat(self.trnMats[behavior_ind])))
    for k in range(0, n_layers):
        ego_embedding = tf.sparse.sparse_dense_matmul(A_hat, ego_embedding)
        all_embeddings.append(ego_embedding)
    all_embeddings = tf.stack(all_embeddings, 1)
    all_embeddings = tf.reduce_mean(
        input_tensor=all_embeddings, axis=1, keepdims=False
    )
    return all_embeddings

In [4]:
def cascading_block(self):
    # TODO: have a look at the variables not used
    self.temlats = list()  # NOT used
    self.translats = list() # NOT used
    ulats = []
    ilats = []
    # ues a dictionary to store the embedding (user || item) for each behavior
    all_embeddings = {}
    # state the tf.Variables [mapping from input space to embedding space]
    UEmbed = NNs.defineParam('UEmbed', shape=[args.user, args.latdim], dtype=tf.float32,
                             reg=True)  # [user * latDim]
    IEmbed = NNs.defineParam('IEmbed', shape=[args.item, args.latdim], dtype=tf.float32,
                             reg=True)  # [item * latDim]
    # Equation (1): concat the user embedding and item embedding
    # [(user + item) * latDim]
    total_embeddings = tf.concat([UEmbed, IEmbed], axis=0)
    for inp in range(args.intTypes):
        layer_embeddings = total_embeddings
        # use LightGCN to embed the input
        layer_embeddings = self._create_lightGCN_embed(layer_embeddings, inp)
        # TODO: add dropout layer here?
        # Embedding Normalization (L2 Norm)
        layer_embeddings = layer_embeddings / (1e-6 + tf.sqrt(1e-6 + tf.reduce_sum(tf.square(layer_embeddings),
                                                                                   axis=-1, keepdims=True)))
        # Residual
        total_embeddings = layer_embeddings + total_embeddings
        all_embeddings[inp] = total_embeddings
        # TODO: data redundancy? the user embedding & item embedding matrix as 2 lists <=> all_embeddings[inp] as a dict
        ulat, ilat = tf.split(
            all_embeddings[inp], [args.user, args.item], 0
        )
        ulats.append(ulat)
        ilats.append(ilat)
    # use the latest [-1] users/items embedding for look-up
    # ulat = FC(ulat[-1], args.latdim, reg=True, useBias=True, name='ablation_trans',
    # 		  activation='relu')
    # ilat = FC(ilats[-1], args.latdim, reg=True, useBias=True, name='ablation_trans', reuse=True,
    # 		  activation='relu')
    pckUlat = tf.nn.embedding_lookup(ulats[-1], self.uids)
    pckIlat = tf.nn.embedding_lookup(ilats[-1], self.iids)
    predLat = pckUlat * pckIlat * args.mult
    for i in range(1):
        predLat = FC(predLat, args.latdim, reg=True, useBias=True, activation='relu') + predLat
    pred = tf.squeeze(FC(predLat, 1, reg=True, useBias=True)) # * args.mult
    return pred

In [6]:
user = 10
item = 8
latdim = 5

In [35]:
UEmbed = tf.random.uniform([user, latdim])
IEmbed = tf.random.uniform([item, latdim])

A_hat = tf.sparse.SparseTensor()
A_hat = tf.random.uniform([item + user, item + user])

In [19]:
init_op = tf.global_variables_initializer()

In [37]:
with tf.Session() as sess:
    sess.run(init_op) #execute init_op
    #print the random values that we sample
    total_embeddings = tf.concat([UEmbed, IEmbed], axis=0)
    all_embeddings = [total_embeddings]
    total_embeddings = tf.sparse.sparse_dense_matmul(A_hat, total_embeddings)
    print(total_embeddings.shape)
    print(sess.run(total_embeddings))
    print(all_embeddings[0].shape)

TypeError: Input must be a SparseTensor.